In [1]:
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

In [2]:

from langdetect import detect
from googletrans import Translator
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM
import random
import torch

# Sprawdź dostępność GPU i wybierz urządzenie
device = 0 if torch.cuda.is_available() else -1
print(f"Używane urządzenie: {'GPU' if device == 0 else 'CPU'}")


c:\Users\Jan\.version-fox\cache\python\current\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Używane urządzenie: GPU


In [3]:
# ZAD 1
# Załaduj pipeline do tłumaczenia En -> Pl
# Model Helsinki-NLP jest popularnym wyborem dla wielu par językowych
translator_en_pl = pipeline("translation_en_to_pl", model="Helsinki-NLP/opus-mt-en-mul", device=device)

# Tekst do przetłumaczenia
text_en = ">>pol<< Hello, world!This is a sample text to be translated."

# Wykonaj tłumaczenie                                           
translation_result = translator_en_pl(text_en)

# Wyświetl wynik
print(f"Tekst oryginalny (EN): {text_en}")
# Wynik jest listą słowników, bierzemy pierwszy element i klucz 'translation_text'
print(f"Tłumaczenie (PL): {translation_result[0]['translation_text']}")

Device set to use cuda:0


Tekst oryginalny (EN): >>pol<< Hello, world!This is a sample text to be translated.
Tłumaczenie (PL): Cześć, świat! To jest tekst próbny, które ma być przetłumaczony.


In [4]:
# ZADD 2
from langdetect import detect

text_unknown_lang = "Bonjour le monde! Ceci est un exemple de texte."

try:
    # Wykryj język źródłowy
    source_language = detect(text_unknown_lang)
    print(f"Wykryty język źródłowy: {source_language}")

    model_name = f"Helsinki-NLP/opus-mt-{source_language}-pl"
    translator_dynamic = pipeline("translation", model=model_name, device=device)

    # Tłumacz
    translation_dynamic_result = translator_dynamic(text_unknown_lang)

    print(f"Tekst oryginalny ({source_language.upper()}): {text_unknown_lang}")
    print(f"Tłumaczenie (PL): {translation_dynamic_result[0]['translation_text']}")


except Exception as e:
    print(f"Wystąpił błąd podczas detekcji lub tłumaczenia: {e}")
    print("Upewnij się, że biblioteka langdetect jest zainstalowana i tekst nie jest zbyt krótki lub niejednoznaczny.")



Wykryty język źródłowy: fr


Device set to use cuda:0


Tekst oryginalny (FR): Bonjour le monde! Ceci est un exemple de texte.
Tłumaczenie (PL): Witaj świat! To jest przykład tekstu.


In [5]:
import asyncio
# Tekst wejściowy
input_text = "Gilgamesh and Enkidu kill the Bull of Heaven, insulting Ishtar in the process, after which the gods decide to sentence Enkidu to death and kill him by giving him a fatal illness."

# 1. Wykryj język źródłowy
try:
    source_lang = detect(input_text)
    print(f"Wykryty język źródłowy: {source_lang}")
except Exception as e:                          
    print(f"Błąd detekcji języka: {e}")
    source_lang = 'en'

translator_google = Translator()
target_languages = ['pl', 'de', 'fr', 'es', 'it']
translations = {}

translations['pl'] = "Gilgamesz i Enkidu zabijają Niebiańskiego Byka, obrażając przy tym Isztar, po czym bogowie decydują się skazać Enkidu na śmierć, zadając mu śmiertelną chorobę."
translations['de'] = "Gilgamesch und Enkidu töten den Himmelsstier und beleidigen dabei Ishtar. Daraufhin beschließen die Götter, Enkidu zum Tode zu verurteilen und ihn durch eine tödliche Krankheit zu töten."
translations['fr'] = "Gilgamesh et Enkidu tuent le Taureau du Ciel, insultant Ishtar au passage, après quoi les dieux décident de condamner Enkidu à mort et de le tuer en lui infligeant une maladie mortelle."
translations['es'] = "Gilgamesh y Enkidu matan al Toro del Cielo, insultando a Ishtar en el proceso, después de lo cual los dioses deciden sentenciar a Enkidu a muerte y matarlo dándole una enfermedad fatal."
translations['it'] = "Gilgamesh ed Enkidu uccidono il Toro Celeste, insultando Ishtar; dopodiché gli dei decidono di condannare a morte Enkidu e di ucciderlo infliggendogli una malattia mortale."


herbert_model_name = "allegro/herbert-base-cased"
fill_mask_pipeline = pipeline('fill-mask', model=herbert_model_name, tokenizer=herbert_model_name, device=device)
herbert_tokenizer = AutoTokenizer.from_pretrained(herbert_model_name)

print("\n--- Maskowanie i Uzupełnianie HERBERT (na tłumaczeniu PL) ---")

polish_translation = translations.get('pl')

if polish_translation:

    tokens = herbert_tokenizer.tokenize(polish_translation)
    token_indices = list(range(len(tokens)))

    # 4. Wstaw token <mask> w losowym miejscu w środkowej części
    if len(tokens) > 3:
        # Wybierz indeks mniej więcej w środku (np. między 25% a 75% długości)
        min_idx = max(1, len(tokens) // 4)
        max_idx = min(len(tokens) - 2, 3 * len(tokens) // 4)
        if max_idx > min_idx:
            mask_index = random.randint(min_idx, max_idx)

            # Zamień token na maskę
            masked_tokens = tokens[:mask_index] + [herbert_tokenizer.mask_token] + tokens[mask_index+1:]

            # Połącz tokeny z powrotem w zdanie dla pipeline'u fill-mask
            # Tokenizer.convert_tokens_to_string() jest bardziej poprawny niż proste join
            masked_sentence = herbert_tokenizer.convert_tokens_to_string(masked_tokens)

            print(f"Oryginalne tłumaczenie (PL): {polish_translation}")
            print(f"Zdanie z maską: {masked_sentence}")

            # 5. Uzupełnij lukę modelem HERBERT
            try:
                filled_results = fill_mask_pipeline(masked_sentence)
                print("\nSugestie uzupełnienia:")
                for result in filled_results:
                    # Wynik zawiera całe zdanie z uzupełnioną maską
                    print(f"- {result['sequence']} (Score: {result['score']:.4f})")
            except Exception as e:
                print(f"Błąd podczas uzupełniania maski: {e}")

        else:
            print("Zdanie jest zbyt krótkie, aby wstawić maskę w środkowej części.")
    else:
         print("Zdanie jest zbyt krótkie do maskowania.")
else:
    print("Nie udało się uzyskać polskiego tłumaczenia do dalszego przetwarzania.")


Wykryty język źródłowy: en


Device set to use cuda:0



--- Maskowanie i Uzupełnianie HERBERT (na tłumaczeniu PL) ---
Oryginalne tłumaczenie (PL): Gilgamesz i Enkidu zabijają Niebiańskiego Byka, obrażając przy tym Isztar, po czym bogowie decydują się skazać Enkidu na śmierć, zadając mu śmiertelną chorobę.
Zdanie z maską: Gilgamesz i Enkidu zabijają Niebiańskiego Byka , obrażając przy tym Isztar , po czym bogowie decydują się skaza<mask>Enkidu na śmierć , zadając mu śmiertelną chorobę .

Sugestie uzupełnienia:
- Gilgamesz i Enkidu zabijają Niebiańskiego Byka , obrażając przy tym Isztar , po czym bogowie decydują się skaza na Enkidu na śmierć , zadając mu śmiertelną chorobę . (Score: 0.1974)
- Gilgamesz i Enkidu zabijają Niebiańskiego Byka , obrażając przy tym Isztar , po czym bogowie decydują się skaza - Enkidu na śmierć , zadając mu śmiertelną chorobę . (Score: 0.0841)
- Gilgamesz i Enkidu zabijają Niebiańskiego Byka , obrażając przy tym Isztar , po czym bogowie decydują się skaza i Enkidu na śmierć , zadając mu śmiertelną chorobę . (Score

In [6]:
# ZADD 4
import Levenshtein

initial_text = "Robię dwutakt, po strzałach spada łuska. Zemsta i zabójstwa, brudne ręce, we krwi bluzka"

# 1. Wykryj język źródłowy
try:
    source_lang = detect(initial_text)
    print(f"Wykryty język źródłowy: {source_lang}")
    if source_lang != 'pl':
        print("Ostrzeżenie: Przykład zakłada tekst po polsku dla łatwiejszego doboru modeli. Dostosuj modele, jeśli język jest inny.")
except Exception as e:
    print(f"Błąd detekcji języka: {e}")
    source_lang = 'pl' # Załóżmy polski w razie błędu
    
    
# 2. Przetłumacz na angielski i polski (jako baseline)
try:
    translator_pl_en = pipeline("translation", model="Helsinki-NLP/opus-mt-pl-en", device=device)
    base_translation_en = translator_pl_en(initial_text)[0]['translation_text']
    print(f"Tłumaczenie bazowe (EN): {base_translation_en}")
except Exception as e:
    print(f"Błąd tłumaczenia bazowego: {e}")
    raise e


# 3. Sekwencja tłumaczeń wtórnych
translation_sequence = [
    ("Helsinki-NLP/opus-mt-pl-en", "en"),
    ("Helsinki-NLP/opus-mt-en-de", "de"),
    ("Helsinki-NLP/opus-mt-de-fr", "fr"),
    ("Helsinki-NLP/opus-mt-fr-es", "es"),
    ("Helsinki-NLP/opus-mt-es-pl", "pl") 
]

num_cycles = 1
iterations = len(translation_sequence) * num_cycles
print(f"\nRozpoczynam {iterations} tłumaczeń wtórnych ({num_cycles} cykl/e)...")

current_text = initial_text
current_lang = source_lang

for i in range(num_cycles):
    print(f"Cykl {i+1}/{num_cycles}")
    for model_name, target_lang in translation_sequence:
        print(f"  Tłumaczenie: {current_lang} -> {target_lang} (model: {model_name})")
        try:
            translator = pipeline("translation", model=model_name, device=device)

            translated_text = translator(current_text)[0]['translation_text']

            current_text = translated_text
            current_lang = target_lang
            print(f"    Wynik ({target_lang}): {current_text[:100]}...")
        except Exception as e:
            print(f"    BŁĄD podczas tłumaczenia modelem {model_name}: {e}")
            print("    Przerwanie sekwencji tłumaczeń.")
            break
    else:
        continue
    break
final_text = current_text
print(f"\nTekst po {iterations} tłumaczeniach: {final_text}")

# 4. Oblicz metrykę Levenshteina
# Porównujemy tekst początkowy z tekstem końcowym (który powinien być w tym samym języku co początkowy)
if current_lang == source_lang:
    distance = Levenshtein.distance(initial_text, final_text)

    max_len = max(len(initial_text), len(final_text))
    normalized_distance = distance / max_len if max_len > 0 else 0

    print(f"\nOdległość Levenshteina między tekstem początkowym a końcowym: {distance}")
    print(f"Znormalizowana odległość: {normalized_distance:.4f}")
    if normalized_distance < 0.1:
        print("Wynik jest bardzo zbliżony do oryginału.")
    elif normalized_distance < 0.3:
        print("Wynik jest dość podobny do oryginału.")
    elif normalized_distance < 0.6:
        print("Wynik różni się znacząco od oryginału.")
    else:
        print("Wynik jest bardzo różny od oryginału.")
else:
    print(f"\nTekst końcowy jest w języku '{current_lang}', a oczekiwano '{source_lang}'. Nie można bezpośrednio porównać metryką Levenshteina.")


Wykryty język źródłowy: pl


Device set to use cuda:0


Tłumaczenie bazowe (EN): I make a two-track, the shell drops after the shots, revenge and murders, dirty hands, bloody blouse.

Rozpoczynam 5 tłumaczeń wtórnych (1 cykl/e)...
Cykl 1/1
  Tłumaczenie: pl -> en (model: Helsinki-NLP/opus-mt-pl-en)


Device set to use cuda:0


    Wynik (en): I make a two-track, the shell drops after the shots, revenge and murders, dirty hands, bloody blouse...
  Tłumaczenie: en -> de (model: Helsinki-NLP/opus-mt-en-de)


...

    Wynik (fr): Je fais une double piste, la coupe tombe après les coups de feu, la vengeance et les meurtres, les m...
  Tłumaczenie: fr -> es (model: Helsinki-NLP/opus-mt-fr-es)


Device set to use cuda:0


    Wynik (es): Estoy haciendo una doble pista, la copa cae después de los disparos, la venganza y los asesinatos, l...
  Tłumaczenie: es -> pl (model: Helsinki-NLP/opus-mt-es-pl)


Device set to use cuda:0


    Wynik (pl): Robię podwójny trop, drink spada po strzelaninie, zemście i morderstwach, brudne ręce, krwawa bluzka...

Tekst po 5 tłumaczeniach: Robię podwójny trop, drink spada po strzelaninie, zemście i morderstwach, brudne ręce, krwawa bluzka.

Odległość Levenshteina między tekstem początkowym a końcowym: 52
Znormalizowana odległość: 0.5149
Wynik różni się znacząco od oryginału.


In [7]:
## ZAD 5 
# 1. Wczytaj tekst z pliku PDF za pomocą PyPDF2
from PyPDF2 import PdfReader

pdf_file_path = 'PRZYKLADOWYPDFEN.pdf'
output_txt_path = 'PRZYKLADOWYPDFEN_pypdf2.txt'

extracted_text = ""
try:
    reader = PdfReader(pdf_file_path)
    print(f"Odczytano plik PDF: {pdf_file_path} (Liczba stron: {len(reader.pages)})")
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        if text:
            print(f"  Odczytano tekst ze strony {i+1}...")
            extracted_text += text + "\n" # Dodaj nową linię między stronami
        else:
            print(f"  Strona {i+1} nie zawierała możliwego do wyekstrahowania tekstu (może być obrazem?).")

except FileNotFoundError:
    print(f"BŁĄD: Plik PDF '{pdf_file_path}' nie został znaleziony.")
    # Zakończ, jeśli plik nie istnieje
    raise
except Exception as e:
    print(f"BŁĄD podczas odczytu pliku PDF: {e}")
    # Zakończ w razie innych błędów odczytu
    raise


Odczytano plik PDF: PRZYKLADOWYPDFEN.pdf (Liczba stron: 1)
  Odczytano tekst ze strony 1...


In [8]:
from transformers import MarianMTModel
source_lang = "eng"
target_lang = "pol"

model_name = f"Allegro/BiDi-{source_lang}-{target_lang}"

translated_text = ""
if extracted_text:
    print("\nRozpoczynanie tłumaczenia...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

        text = f">>{target_lang}<<" + " " + extracted_text

        batch_to_translate = [text]
        translations = model.generate(**tokenizer.batch_encode_plus(batch_to_translate, return_tensors="pt"))
        translated_text = tokenizer.batch_decode(translations, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]

        print("Tłumaczenie zakończone.")

    except Exception as e:
        print(f"BŁĄD podczas tłumaczenia: {e}")

# 3. Zapisz wynik do pliku tekstowego
if translated_text:
    try:
        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(translated_text)
        print(f"\nPrzetłumaczony tekst został zapisany do pliku: {output_txt_path}")
        print("\n--- Przetłumaczony tekst (początek) ---")
        print(translated_text[:500] + "...")
        print("--- Koniec fragmentu ---")
    except Exception as e:
        print(f"BŁĄD podczas zapisu do pliku '{output_txt_path}': {e}")
elif not extracted_text:
     print("\nNie zapisano wyniku, ponieważ nie udało się wyekstrahować tekstu z PDF.")
else:
     print("\nNie zapisano wyniku, ponieważ wystąpił błąd podczas tłumaczenia.")


Rozpoczynanie tłumaczenia...
Tłumaczenie zakończone.

Przetłumaczony tekst został zapisany do pliku: PRZYKLADOWYPDFEN_pypdf2.txt

--- Przetłumaczony tekst (początek) ---
Jest to pierwsza strona przykładowego dokumentu PDF. Zawiera on jakiś tekst w języku angielskim. Rozpakujemy ten tekst za pomocą PyPDF2 i przetłumaczymy go. Dodajmy kolejne zdanie za dobrą miarę. Naturalne przetwarzanie języka jest fascynujące....
--- Koniec fragmentu ---


In [27]:
## ZAD 6

from transformers import pipeline, MBartForConditionalGeneration, MBart50TokenizerFast
import concurrent.futures
import time

input_file_path = 'sacred_theory_of_the_earth.txt'
output_markdown_path = 'translated_sacred_theory_of_the_earth.md'
sentences_per_chunk = 3  # Ile zdań ma tworzyć jeden fragment do tłumaczenia

# Komórka 5: Funkcja do wczytywania i dzielenia tekstu na fragmenty
def load_and_chunk_text(filepath, num_sentences_per_chunk=3):
    """
    Wczytuje tekst z pliku i dzieli go na fragmenty.
    Każdy fragment składa się z określonej liczby zdań.
    """
    print(f"Wczytywanie i dzielenie tekstu z: {filepath}")
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text_content = f.read()
    except FileNotFoundError:
        print(f"BŁĄD: Plik '{filepath}' nie został znaleziony.")
        return []
    
    # Dzielenie tekstu na zdania za pomocą NLTK
    sentences = nltk.sent_tokenize(text_content)
    
    chunks = []
    current_chunk_sentences = []
    for i in range(0, len(sentences), num_sentences_per_chunk):
        chunk = " ".join(sentences[i:i + num_sentences_per_chunk])
        chunks.append(chunk)
        
    print(f"Podzielono tekst na {len(chunks)} fragmentów (po {num_sentences_per_chunk} zdania/fragment).")
    return chunks


In [18]:
# Inincjalizacja mBart
model_name = "facebook/mbart-large-50-many-to-many-mmt"
print(f"Ładowanie modelu: {model_name}. To może potrwać chwilę...")

try:
    tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
    model = MBartForConditionalGeneration.from_pretrained(model_name)
    

    translator = pipeline(
        'translation',
        model=model,
        tokenizer=tokenizer,
        src_lang="en_XX",  # Język źródłowy (angielski)
        tgt_lang="pl_PL",  # Język docelowy (polski)
        device=device,
        max_length=512     # Maksymalna długość generowanego tłumaczenia (można dostosować)
    )
    print("Model i translator zainicjalizowane pomyślnie.")
except Exception as e:
    print(f"Wystąpił błąd podczas inicjalizacji modelu: {e}")
    print("Upewnij się, że masz połączenie z internetem przy pierwszym uruchomieniu, aby pobrać model.")
    translator = None # Ustawiamy na None, aby dalsza część kodu nie próbowała użyć niezainicjalizowanego translatora

Ładowanie modelu: facebook/mbart-large-50-many-to-many-mmt. To może potrwać chwilę...


Device set to use cuda:0


Model i translator zainicjalizowane pomyślnie.


In [24]:
# Komórka 7: Funkcja do tłumaczenia pojedynczego fragmentu
def translate_single_chunk(text_chunk):
    """
    Tłumaczy pojedynczy fragment tekstu używając zainicjalizowanego translatora.
    """
    if not translator or not text_chunk:
        return "[Błąd: Translator nie jest dostępny lub fragment jest pusty]"
    
    try:
        translation_result = translator(text_chunk)
        
        if translation_result and isinstance(translation_result, list) and 'translation_text' in translation_result[0]:
            return translation_result[0]['translation_text']
        else:
            return "[Błąd: Nie udało się przetłumaczyć fragmentu]"
            
    except Exception as e:
        print(f"Błąd podczas tłumaczenia fragmentu: '{text_chunk[:50]}...'. Błąd: {e}")
        return f"[Błąd tłumaczenia: {e}]"

In [28]:
#Wielowątkowe przetwarzania tekstu
if translator: # Kontynuuj tylko jeśli translator został poprawnie załadowany
    text_chunks_to_translate = load_and_chunk_text(input_file_path, sentences_per_chunk)
    
    if text_chunks_to_translate:
        translated_chunks_results = [None] * len(text_chunks_to_translate) # Do przechowywania wyników w kolejności
        
        num_workers = 4
        
        print(f"\nRozpoczynanie tłumaczenia {len(text_chunks_to_translate)} fragmentów przy użyciu {num_workers} wątków...")
        start_time = time.time()
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            # Mapowanie przyszłych zadań na indeksy, aby zachować kolejność
            future_to_index = {
                executor.submit(translate_single_chunk, chunk): i 
                for i, chunk in enumerate(text_chunks_to_translate)
            }
            processed_count = 0
            for i, future in enumerate(concurrent.futures.as_completed(future_to_index)):
                original_index = future_to_index[future]
                try:
                    translated_text = future.result()
                    translated_chunks_results[original_index] = translated_text
                    print(f"Przetłumaczono fragment {original_index + 1}/{len(text_chunks_to_translate)}")
                except Exception as exc:
                    print(f"Fragment {original_index + 1} wygenerował błąd: {exc}")
                    translated_chunks_results[original_index] = "[Błąd podczas pobierania wyniku tłumaczenia]"
                processed_count += 1
                if processed_count % 10 == 0 or processed_count == len(text_chunks_to_translate):
                     print(f"Przetworzono {processed_count}/{len(text_chunks_to_translate)} fragmentów...")

        end_time = time.time()
        total_translation_time = end_time - start_time
        print(f"\nZakończono tłumaczenie wszystkich fragmentów.")
        print(f"Całkowity czas tłumaczenia: {total_translation_time:.2f} sekund.")

    else:
        print("Nie znaleziono fragmentów do tłumaczenia.")
        translated_chunks_results = []
else:
    print("Translator nie został poprawnie załadowany. Pomijanie tłumaczenia.")
    translated_chunks_results = []
    text_chunks_to_translate = []

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Wczytywanie i dzielenie tekstu z: sacred_theory_of_the_earth.txt
Podzielono tekst na 2068 fragmentów (po 3 zdania/fragment).

Rozpoczynanie tłumaczenia 2068 fragmentów przy użyciu 4 wątków...
Błąd podczas tłumaczenia fragmentu: 'Title: The sacred theory of the Earth, Volume 2 (o...'. Błąd: Already borrowed
Błąd podczas tłumaczenia fragmentu: 'The TWO LAST BOOKS,

                 _Concerning ...'. Błąd: Already borrowed
Błąd podczas tłumaczenia fragmentu: 'TO THE
                                QUEEN’S
   ...'. Błąd: Already borrowed
Błąd podczas tłumaczenia fragmentu: 'They both indeed agree in this, that there is a
WO...'. Błąd: Already borrowed
Błąd podczas tłumaczenia fragmentu: 'Cities that are burnt, are commonly rebuilt more b...'. Błąd: Already borrowed
Błąd podczas tłumaczenia fragmentu: 'But, I fear, it may be thought no very proper Addr...'. Błąd: Already borrowed
Przetłumaczono fragment 7/2068
Przetłumaczono fragment 2/2068
Przetłumaczono fragment 3/2068
Przetłumaczono frag

In [30]:
import os
# Generowanie markdowna
if translated_chunks_results:
    print(f"\nZapisywanie przetłumaczonych fragmentów do pliku Markdown: {output_markdown_path}")
    
    with open(output_markdown_path, 'w', encoding='utf-8') as md_file:
        md_file.write(f"# Tłumaczenie dokumentu: '{os.path.basename(input_file_path)}'\n\n")
        md_file.write(f"Liczba zdań na fragment: {sentences_per_chunk}\n")
        md_file.write("---\n\n")
        
        for i, (original_chunk, translated_chunk) in enumerate(zip(text_chunks_to_translate, translated_chunks_results)):
            md_file.write(f"## Fragment {i+1}\n\n")
            
            md_file.write(f"**Oryginał (Angielski):**\n")
            md_file.write(f"```text\n{original_chunk.strip()}\n```\n\n")
            
            md_file.write(f"**Tłumaczenie (Polski):**\n")
            md_file.write(f"```text\n{translated_chunk.strip()}\n```\n\n")
            
            md_file.write("---\n\n")
            
    print(f"Wynik zapisano pomyślnie w '{output_markdown_path}'.")
elif translator:
    print("Brak przetłumaczonych fragmentów do zapisania.")
else: 
    print("Proces tłumaczenia nie został przeprowadzony z powodu problemów z modelem. Nic nie zapisano.")

if translated_chunks_results and text_chunks_to_translate:
    print("\n--- Przykładowe przetłumaczone fragmenty ---")
    for i in range(min(3, len(translated_chunks_results))):
        print(f"\nFragment {i+1}:")
        print(f"Oryginał: {text_chunks_to_translate[i][:100]}...")
        print(f"Tłumaczenie: {translated_chunks_results[i][:100]}...")
else:
    print("\nBrak wyników do wyświetlenia.")
    



Zapisywanie przetłumaczonych fragmentów do pliku Markdown: translated_sacred_theory_of_the_earth.md
Wynik zapisano pomyślnie w 'translated_sacred_theory_of_the_earth.md'.

--- Przykładowe przetłumaczone fragmenty ---

Fragment 1:
Oryginał: ﻿The Project Gutenberg eBook of The sacred theory of the Earth, Volume 2 (of 2)
    
This ebook is f...
Tłumaczenie: The Project Gutenberg eBook of The sacred theory of the Earth, Volume 2 (of 2) Ten e-book jest do uż...

Fragment 2:
Oryginał: Title: The sacred theory of the Earth, Volume 2 (of 2)
        Containing an account of the original...
Tłumaczenie: [Błąd tłumaczenia: Already borrowed]...

Fragment 3:
Oryginał: The TWO LAST BOOKS,

                 _Concerning the Burning of the WORLD,
                        ...
Tłumaczenie: [Błąd tłumaczenia: Already borrowed]...


In [48]:
# Naprawa markdowna
import re
TRANSLATION_ERROR_PREFIX = "[Błąd tłumaczenia:"
def regenerate_failed_translations_in_markdown(markdown_file_path, translator_pipeline_instance):
    """
    Odczytuje plik Markdown, znajduje fragmenty z błędami tłumaczenia,
    i próbuje je przetłumaczyć ponownie.
    """
    if not translator_pipeline_instance:
        print("BŁĄD KRYTYCZNY: Translator nie jest dostępny dla funkcji regeneracji.")
        return

    print(f"\nRozpoczynanie próby naprawy błędów w pliku: {markdown_file_path}")
    try:
        with open(markdown_file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except FileNotFoundError:
        print(f"BŁĄD: Plik Markdown '{markdown_file_path}' nie został znaleziony do naprawy.")
        return
    fragment_block_pattern = re.compile(
        r"((?:## Fragment \d+\s*\n\n\*\*Oryginał \(Angielski\):\*\*\s*\n```text\n(.*?)\n```\s*\n\n\*\*Tłumaczenie \(Polski\):\*\*\s*\n```text\n))" 
        r"(.*?)"  
        r"((?:```\s*\n\n---))",
        re.DOTALL
    )
    
    fragments_retried_in_regen = 0
    fragments_corrected_in_regen = 0

    def replacement_function(match):
        nonlocal fragments_retried_in_regen, fragments_corrected_in_regen
        
        prefix_until_polish_translation_content = match.group(1)
        original_english_text_content = match.group(2).strip()
        current_polish_translation_content = match.group(3)
        suffix_after_polish_translation_content = match.group(4)
        
        # Usuwamy białe znaki z początku/końca dla porównania, ale zachowujemy oryginalne formatowanie, jeśli to możliwe
        current_polish_translation_content_stripped = current_polish_translation_content.strip()

        if current_polish_translation_content_stripped.startswith(TRANSLATION_ERROR_PREFIX):
            fragments_retried_in_regen += 1
            print(f"Regeneracja: Znaleziono błąd w fragmencie. Oryginał: '{original_english_text_content[:60]}...'")
            print("Regeneracja: Próba ponownego tłumaczenia (sekwencyjnie)...")
            
            new_translation_content = translate_single_chunk(original_english_text_content) # Używa globalnego translatora
            
            if not new_translation_content.startswith(TRANSLATION_ERROR_PREFIX):
                print(f"Regeneracja: Poprawiono tłumaczenie: '{new_translation_content[:60]}...'")
                fragments_corrected_in_regen += 1
                # Zwracamy zrekonstruowany blok z nowym tłumaczeniem, zachowując \n jeśli było
                # Jeśli oryginalny current_polish_translation_content miał \n na końcu, nowy też powinien
                if current_polish_translation_content.endswith('\n') and not new_translation_content.endswith('\n'):
                    new_translation_to_insert = new_translation_content + '\n'
                elif not current_polish_translation_content.endswith('\n') and new_translation_content.endswith('\n'):
                     new_translation_to_insert = new_translation_content.rstrip('\n')
                else:
                    new_translation_to_insert = new_translation_content

                return f"{prefix_until_polish_translation_content}{new_translation_to_insert}{suffix_after_polish_translation_content}"
            else:
                print(f"Regeneracja: Ponowne tłumaczenie również nie powiodło się dla: '{original_english_text_content[:60]}...'")
                return match.group(0) 
        else:
            return match.group(0)

    if translator:
        final_content = fragment_block_pattern.sub(replacement_function, content)
    else:
        print("Regeneracja niemożliwa - translator nie jest załadowany.")
        final_content = content

    if fragments_retried_in_regen > 0:
        if content != final_content:
            with open(markdown_file_path, 'w', encoding='utf-8') as f:
                f.write(final_content)
            print(f"Zakończono regenerację. Próbowano dla {fragments_retried_in_regen} fragmentów, poprawiono {fragments_corrected_in_regen}.")
        else:
            print(f"Zakończono regenerację. Próbowano dla {fragments_retried_in_regen} fragmentów, ale nic się nie zmieniło w pliku.")
    else:
        print(f"Zakończono regenerację. Nie znaleziono fragmentów oznaczonych jako błędne w pliku '{markdown_file_path}'.")


In [49]:
# Wywołanie naprawy
if translated_chunks_results and os.path.exists(output_markdown_path) and translator:
    regenerate_failed_translations_in_markdown(output_markdown_path, translator) 
elif not translator:
     print("Pominięto regenerację pliku Markdown, ponieważ translator nie jest dostępny.")
else:
    print("Pominięto regenerację pliku Markdown (brak wyników lub plik nie istnieje).")


Rozpoczynanie próby naprawy błędów w pliku: translated_sacred_theory_of_the_earth.md
Regeneracja: Znaleziono błąd w fragmencie. Oryginał: 'Title: The sacred theory of the Earth, Volume 2 (of 2)
     ...'
Regeneracja: Próba ponownego tłumaczenia (sekwencyjnie)...
Regeneracja: Poprawiono tłumaczenie: 'Tytuł: The sacred theory of the Earth, Volume 2 (of 2) Conta...'
Regeneracja: Znaleziono błąd w fragmencie. Oryginał: 'The TWO LAST BOOKS,

                 _Concerning the Burnin...'
Regeneracja: Próba ponownego tłumaczenia (sekwencyjnie)...
Regeneracja: Poprawiono tłumaczenie: 'TWO LAST BOOKS, _Concerning the Burning of the WORLD, AND Co...'
Regeneracja: Znaleziono błąd w fragmencie. Oryginał: 'TO THE
                                QUEEN’S
             ...'
Regeneracja: Próba ponownego tłumaczenia (sekwencyjnie)...
Regeneracja: Poprawiono tłumaczenie: 'WSZYSTKIEJ WSZYSTKIEJ MAJESTY Królowej. MADAM, mając zaszczy...'
Regeneracja: Znaleziono błąd w fragmencie. Oryginał: 'They both indeed 

In [51]:
# Zad 7
if extracted_text:
    print("--- Oryginalny tekst z PDF ---")
    print(extracted_text)
else:
    print(f"Nie udało się wczytać tekstu z {pdf_file_path}. Upewnij się, że plik istnieje i jest poprawnym PDFem.")


--- Oryginalny tekst z PDF ---
This is the first page of the example PDF document. 
It contains some text in English.
We will extract this text using PyPDF2 and translate it.
Let's add another sentence for good measure.
Natural Language Processing is fascinating.



In [64]:
# Inicjalizacja modelu i tokenizera plt5

from transformers import AutoModelForSeq2SeqLM

tokenizer_plt5 = AutoTokenizer.from_pretrained("allegro/plt5-base")
model_plt5 = AutoModelForSeq2SeqLM.from_pretrained("allegro/plt5-base")

def translate_with_plt5(text, tone="neutralny"):

    if tone == "formalny":
        prompt = f"Przetłumacz z angielskiego na polski w formalnym stylu: {text}"
    elif tone == "potoczny":
        prompt = f"Przetłumacz z angielskiego na polski w potocznym stylu: {text}"
    else: # neutralny
        prompt = f"Przetłumacz z angielskiego na polski: {text}"

    try:
        inputs = tokenizer_plt5(prompt, return_tensors="pt", padding="longest", truncation=True, max_length=512)
        output_sequences = model_plt5.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=512,
            num_beams=5,
            early_stopping=True
        )
        translated_text = tokenizer_plt5.decode(output_sequences[0], skip_special_tokens=True)
        return translated_text
    except Exception as e:
        print(f"Błąd podczas tłumaczenia z plt5: {e}")
        return None

In [66]:
if extracted_text:
    print("\nWybierz ton tłumaczenia:")
    print("1. Formalny")
    print("2. Potoczny")
    print("3. Neutralny (domyślny)")
    
    user_choice = input("Wybierz numer (1-3): ")
    selected_tone = "neutralny"
    if user_choice == "1":
        selected_tone = "formalny"
    elif user_choice == "2":
        selected_tone = "potoczny"

    print(f"\n--- Tłumaczenie (plt5 - ton: {selected_tone}) ---")
    translated_text_plt5 = translate_with_plt5(extracted_text, selected_tone)
    if translated_text_plt5:
        print(translated_text_plt5)

        # Zapis wyniku do pliku tekstowego
        output_filename_plt5 = f"tlumaczenie_plt5_{selected_tone}.txt"
        try:
            with open(output_filename_plt5, "w", encoding="utf-8") as f:
                f.write(translated_text_plt5)
            print(f"\nZapisano tłumaczenie do pliku: {output_filename_plt5}")
        except Exception as e:
            print(f"Błąd podczas zapisywania pliku: {e}")
    else:
        print("Nie udało się przetłumaczyć tekstu za pomocą plt5.")


Wybierz ton tłumaczenia:
1. Formalny
2. Potoczny
3. Neutralny (domyślny)

--- Tłumaczenie (plt5 - ton: formalny) ---
規. Przetłumacz z angielskiego na polski w formalnym stylu:

Zapisano tłumaczenie do pliku: tlumaczenie_plt5_formalny.txt
